In [1]:
from scipy.stats import pearsonr
import pandas as pd
import random 
from scipy.stats import describe, pearsonr, zscore, f_oneway, yeojohnson, shapiro, probplot, levene
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

def explore(df: pd.DataFrame) -> pd.DataFrame:
    ex1, ex2, ex3 = random.sample(range(len(df)), 3)

    print("Dataframe total rows: ", len(df))
    df_info = pd.DataFrame(data = df.dtypes)
    not_missing_values_total = df.notnull().sum()
    not_missing_values_percent = round(not_missing_values_total/len(df)*100,2).astype(str)+" %"

    return pd.concat([df_info[0].rename("Data Type"),\
            df.T[ex1].rename("Example 1"),\
            df.T[ex2].rename("Example 2"),\
            df.T[ex3].rename("Example 3"),\
            not_missing_values_total.rename("Total Not Missing"), \
            not_missing_values_percent.rename("% of not missing values")], axis=1)
    
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included    

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [2]:
import saspy
sas_session = saspy.SASsession()
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 1115



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /home/armando/.virtualenvs/school-54vk9BCB/lib/python3.8/site-packages/saspy/sascfg_personal.py
WORK Path             = /saswork/SAS_workF6B90001A622_odaws01-usw2.oda.sas.com/SAS_work52710001A622_odaws01-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.0
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 108066


In [3]:
%%SAS sas_session

libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

In [4]:
import pandas as pd

# #comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

# data1 = sas_session.sasdata2dataframe(
# table='hist',
# libref='cortex'
# )

# data2 = sas_session.sasdata2dataframe(
# table='target_rd2',
# libref='cortex'
# )

## Merge the Data

In [5]:
#Step1 Merge the Data
# data_merge = pd.merge(data1, data2, on=["ID"],how="right")
# data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
# data_merge.to_csv("og2.csv", index= False)
data_merge = pd.read_csv("og2.csv")
data_merge.head()

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
0,2000004.0,LEE,MARY,1.0,78.0,23700.0,High School,Rural,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,20.0
1,2000008.0,STINSON,MICHAEL,0.0,30.0,92700.0,University / College,Suburban,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,30.0
2,2000012.0,GUALTIERI,MARIA,1.0,52.0,77600.0,High School,City,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,20.0
3,2000017.0,OLAGUE,DONNA,1.0,50.0,13000.0,High School,Suburban,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1.0,20.0,1.0,1.0,75.0
4,2000021.0,MCCASKILL,DENISE,1.0,39.0,237800.0,University / College,Rural,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,20.0


In [6]:
explore(data_merge)

Dataframe total rows:  149457


,Data Type,Example 1,Example 2,Example 3,Total Not Missing,% of not missing values
ID,float64,2584309.0,2281634.0,2055992.0,149457,100.0 %
LastName,object,TIPPING,HOWARD,NIGH,149454,100.0 %
FirstName,object,JANET,THOMAS,KENNETH,149457,100.0 %
Woman,float64,1.0,0.0,0.0,149457,100.0 %
Age,float64,56.0,27.0,67.0,149457,100.0 %
Salary,float64,3600.0,53200.0,39900.0,149457,100.0 %
Education,object,University / College,University / College,High School,149457,100.0 %
City,object,City,Suburban,Downtown,149457,100.0 %
SeniorList,float64,0.0,4.0,2.0,149457,100.0 %
NbActivities,float64,0.0,0.0,0.0,149457,100.0 %


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [7]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.
var_types = {"number" : "median", "object" : "mode"}
for var_type in list(var_types.keys()):
    for col in data_merge.select_dtypes(include=var_type).columns:
        data_merge[col] = data_merge[col].fillna(getattr(data_merge[col], var_types[var_type])())
            
for col in data_merge.select_dtypes(include="object"):
    data_merge[col] = data_merge[col].astype('category')
    data_merge[col] = data_merge[col].cat.codes

In [8]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.feature_selection import SelectFromModel
# sel = SelectFromModel(RandomForestRegressor())
# X = data_merge.drop(["ID", "AmtThisYear", "GaveThisYear", "LastName", "FirstName"], axis=1)
# sel.fit(X, data_merge["AmtThisYear"])

# importances = sel.estimator_.feature_importances_
# names = sel.estimator_.feature_names_in_

# indices = np.argsort(importances)[::-1]

# plt.figure(figsize=(10,5))
# plt.title("Feature importances")
# plt.bar(range(X.shape[1]), importances[indices], color="r", align="center")
# plt.xticks(range(X.shape[1]), [names[i] for i in indices])
# plt.xlim([-1, X.shape[1]])
# plt.show()

In [9]:
from sklearn.preprocessing import PowerTransformer

power = PowerTransformer(method='yeo-johnson', standardize=True)
X= data_merge.drop(["ID", "AmtThisYear"], axis=1)
X_cols = X.columns
X = power.fit_transform(X)
X = pd.DataFrame(X, columns = X_cols) 

In [10]:
selection = ['Salary', 'Age', 'SeniorList','TotalGift', 'City', 'Contact', 'Referrals', 'Woman','MinGift','NbActivities']

cols = selection +  ["AmtThisYear"] 

In [11]:
data = X
data["AmtThisYear"] = data_merge["AmtThisYear"]

data = data[cols] 
data = data[(np.abs(zscore(data.select_dtypes(include ='number'))) < 3).all(axis=1)]
len(data)

146148

## Data Partition

In [12]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.4,random_state=5678) # you can change the percentage
train.sample(2)

,Salary,Age,SeniorList,TotalGift,City,Contact,Referrals,Woman,MinGift,NbActivities,AmtThisYear
71515,0.453155,-0.382007,0.322972,-1.102681,-0.196442,-0.59473,-0.86165,0.913746,0.546534,0.698454,20.0
111863,0.094672,-0.823661,-0.251501,0.007054,1.165399,-0.59473,-0.86165,0.913746,0.041151,-0.891909,50.0



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [13]:
from sklearn import linear_model

X_train = train.drop("AmtThisYear", axis=1)
Y_train = train['AmtThisYear']
X_valid = validation.drop("AmtThisYear", axis=1)
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[59.35946199 52.3671434  43.80583353 ... 44.73500064 53.5420018
 52.00867468]


In [14]:
# #you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

36.51669885462472
4892.630801589629
69.94734306311877


## Regression Tree Model（Py）

In [15]:
from sklearn.tree import DecisionTreeRegressor

X_train = train.drop("AmtThisYear", axis=1)
Y_train = train['AmtThisYear']
X_valid = validation.drop("AmtThisYear", axis=1)
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[53.3794855  63.76623377 41.04275093 ... 37.70411392 47.02126597
 49.42248347]


In [16]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

36.48043149725757
4884.448350092155
69.88882850708083


In [17]:
X_train = train.drop("AmtThisYear", axis=1)
Y_train = train['AmtThisYear']
X_valid = validation.drop("AmtThisYear", axis=1)
Y_valid = validation['AmtThisYear']

In [18]:
import xgboost as xgb
import numpy as np
import optuna
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from optuna.integration import XGBoostPruningCallback
from sklearn.model_selection import RepeatedKFold
from optuna import create_study

def objective(
    trial,
    X,
    y,
    random_state=124,
    n_splits=10,
    n_repeats=2,
    n_jobs=12,
    early_stopping_rounds=100,
):
    # XGBoost parameters
    params = {
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": 300,
        "max_depth": trial.suggest_int("max_depth", 5, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.00001, 0.01),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
    }

    model = XGBRegressor(**params)
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")
    rkf = RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
    )
    X_values = X.values
    y_values = y.values
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_A, X_B = X_values[train_index, :], X_values[test_index, :]
        y_A, y_B = y_values[train_index], y_values[test_index]
        model.fit(
            X_A,
            y_A,
            eval_set=[(X_B, y_B)],
            eval_metric="rmse",
            verbose=0,
            callbacks=[pruning_callback],
            early_stopping_rounds=early_stopping_rounds,
        )
        y_pred[test_index] += model.predict(X_B)
    y_pred /= n_repeats
    return np.sqrt(mean_squared_error(Y_train, y_pred))

/home/armando/.virtualenvs/school-54vk9BCB/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
sampler = TPESampler(seed=124, multivariate=True)
study = create_study(direction="minimize", sampler=sampler)
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        Y_train,
        random_state=124,
        n_splits=10,
        n_repeats=2,
        n_jobs=12,
        early_stopping_rounds=100,
    ),
    n_trials=2,
    n_jobs=12,
)

# display params
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

/home/armando/.virtualenvs/school-54vk9BCB/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:281: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-12-02 07:48:00,975] A new study created in memory with name: no-name-7c7f9464-4357-4db5-8e52-9572cc51d4cd
/tmp/ipykernel_2325/3194058148.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.00001, 0.01),
/tmp/ipykernel_2325/3194058148.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "colsample_bytree": trial.suggest_loguniform("colsample_b

: 

: 

In [ ]:
hp["verbosity"] = 0
hp["objective"] = "reg:squarederror"
hp["n_estimators"] = 300
hp["seed"] = 124
hp["n_jobs"] = 12
model = XGBRegressor(**hp)
rkf = RepeatedKFold(n_splits=10, n_repeats=2, random_state=124)
X_values = X_train.values
y_values = Y_train.values
y_pred = np.zeros_like(Y_valid.values)
for train_index, test_index in rkf.split(X_values):
    X_A, X_B = X_values[train_index, :], X_values[test_index, :]
    y_A, y_B = y_values[train_index], y_values[test_index]
    model.fit(
        X_A,
        y_A,
        eval_set=[(X_B, y_B)],
        eval_metric="rmse",
        early_stopping_rounds=100,
        verbose=0,
    )
    y_pred += model.predict(X_valid.values)
y_pred /= 1 * 10

/home/armando/.virtualenvs/school-54vk9BCB/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/armando/.virtualenvs/school-54vk9BCB/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [ ]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,y_pred))
#MSE
print(metrics.mean_squared_error(Y_valid,y_pred))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,y_pred)))

44.55491734102159
6929.545088651127
83.24388919705234


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [4]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [6]:
#scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")
#scoring_data_contact.to_csv("og2_predict.csv", index= False)
scoring_data_contact = pd.read_csv("og2_predict.csv")

In [17]:

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.


var_types = {"number" : "median", "object" : "mode"}
for var_type in list(var_types.keys()):
    for col in scoring_data_contact.select_dtypes(include=var_type).columns:
        scoring_data_contact[col] = scoring_data_contact[col].fillna(getattr(scoring_data_contact[col], var_types[var_type])())

for col in scoring_data_contact.select_dtypes(include="object"):
    scoring_data_contact[col] = scoring_data_contact[col].astype('category')
    scoring_data_contact[col] = scoring_data_contact[col].cat.codes

X = scoring_data_contact[selection] 


power = PowerTransformer(method='yeo-johnson', standardize=True)
X_cols = X.columns
X = power.fit_transform(X)
X = pd.DataFrame(X, columns = X_cols) 



regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,51.227173
1,2000002.0,46.634160
2,2000003.0,47.507045
3,2000004.0,39.689773
4,2000005.0,48.009611


### Predict 'amount given' for members who were not contacted

In [7]:
#scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")
#scoring_data_nocontact.to_csv("og2_predict_2.csv", index= False)
scoring_data_nocontact = pd.read_csv("og2_predict_2.csv")

In [18]:

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

var_types = {"number" : "median", "object" : "mode"}
for var_type in list(var_types.keys()):
    for col in scoring_data_nocontact.select_dtypes(include=var_type).columns:
        scoring_data_nocontact[col] = scoring_data_nocontact[col].fillna(getattr(scoring_data_nocontact[col], var_types[var_type])())

for col in scoring_data_nocontact.select_dtypes(include="object"):
    scoring_data_nocontact[col] = scoring_data_nocontact[col].astype('category')
    scoring_data_nocontact[col] = scoring_data_nocontact[col].cat.codes

X = scoring_data_nocontact[selection] 


power = PowerTransformer(method='yeo-johnson', standardize=True)
X_cols = X.columns
X = power.fit_transform(X)
X = pd.DataFrame(X, columns = X_cols) 

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,51.227173
1,2000002.0,46.634160
2,2000003.0,47.507045
3,2000004.0,39.689773
4,2000005.0,48.009611


In [19]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,51.227173,51.227173
1,2000002.0,46.634160,46.634160
2,2000003.0,47.507045,47.507045


## Exporting Results to a CSV File

In [20]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.